In [1]:
import pandas as pd
import string

In [2]:
df=pd.read_csv('spam.csv',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
df=df.drop(["Unnamed: 2","Unnamed: 3", "Unnamed: 4"],axis=1)
df.columns=["label","mail"]
df["label"]=df["label"].map({'ham':0,'spam':1})
df.head()

,label,mail
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
def preprocess(mail):
    #removed the punctuations and lowercased the text
    mail="".join([t.lower() for t in mail if t not in string.punctuation])

    #tokenize
    tokens=mail.split(" ") #tokenize in case of whitespace

    #removing the stopwords hahaha
    return " ".join([t for t in tokens if t not in ENGLISH_STOP_WORDS])

df["mail"]=df["mail"].apply(lambda mail: preprocess(mail))
df.head()




,label,mail
0,0,jurong point crazy available bugis n great wor...
1,0,ok lar joking wif u oni
2,1,free entry 2 wkly comp win fa cup final tkts 2...
3,0,u dun say early hor u c say
4,0,nah dont think goes usf lives


In [5]:
X=df["mail"]
y=df["label"]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_vectors=tfidf.fit_transform(X)

In [7]:
X_vectors

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 42907 stored elements and shape (5572, 9229)>

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, y, test_size=0.2, random_state=42)
print(X_train.shape)
print(y_train.shape)

(4457, 9229)
(4457,)


In [9]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [10]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 1], shape=(1115,))

In [11]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.967713004484305
Confusion Matrix:
 [[965   0]
 [ 36 114]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.76      0.86       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.97      1115



In [13]:
from joblib import dump
dump(model, 'spam_classifier_model.joblib')
dump(tfidf, 'tfidf_vectorizer.joblib')

['tfidf_vectorizer.joblib']